# Easy use

This file allows you to use the entire software package yourself, with minimal programming!

In [1]:
%matplotlib inline
import sys
import os

# Sort the paths out to run from this file
parent_path = os.path.dirname(os.getcwd())
src_path = os.path.join(os.path.sep,parent_path, 'src')
sys.path.append(src_path)
print(src_path)

/home/dharrington/cyclops/src


## Step 1 - simulation to csv


In [ ]:
from read_exodus import *



reader = ExodusReader('monoblock_out.e')
pickle_manager = PickleManager()

sensor_region = 'right'
pos_3D = reader.read_pos(sensor_region)
temps = reader.read_scalar(sensor_region, 'temperature')

disp = np.array([
    reader.read_scalar(sensor_region, 'disp_x'),
    reader.read_scalar(sensor_region, 'disp_y'),
    reader.read_scalar(sensor_region, 'disp_z')
]).T

pos_2D = compress_2D(pos_3D)
bounds = find_bounds(pos_2D)
grid = generate_grid(bounds, 30, 30)

temp_field = ScalarField(LModel, bounds, 2)
temp_field.fit_model(pos_2D, temps)

disp_field = VectorField(LModel, bounds, 2)
disp_field.fit_model(pos_2D, disp)

pickle_manager.save_file('simulation', 'field_temp.obj', temp_field)
pickle_manager.save_file('simulation', 'field_disp.obj', disp_field)
pickle_manager.save_file('simulation', 'grid.obj', grid)